In [4]:
from fastapi import FastAPI
from pydantic import BaseModel
from typing import List, Optional
from langchain_community.tools.tavily_search import TavilySearchResults
import os
from langgraph.prebuilt import create_react_agent
from langchain_groq import ChatGroq

In [8]:
import os

groq_api_key = os.getenv("GROQ_API_KEY")
os.environ["TAVILY_API_KEY"] = "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"

MODEL_NAMES = [
    "llama3-70b-8192",
    "mixtral-8x7b-32768"
]

tool_tavily = TavilySearchResults(max_results=2, api_key=os.getenv("TAVILY_API_KEY"))

tools = [tool_tavily]

In [9]:
app = FastAPI(title="Langraph AI Agent")

class RequestState(BaseModel):
    model_name: str
    system_prompt: str 
    messages: List[str]


@app.post("/chat")
def chat_endpoint(request: RequestState):
    """
    API endpoint to interact with the chatbot using Langraph and tools.
    Dynamically selects the model based on user input.
    """

    if request.model_name not in MODEL_NAMES:
        return {"error": f"Model '{request.model_name}' is not supported. Choose from {MODEL_NAMES}."}

    llm = ChatGroq(model_name=request.model_name, api_key=groq_api_key)

    agent = create_react_agent(
        llm=llm,
        tools=tools,
        system_message=request.system_prompt
    )

    state = {"messages": request.messages}

    result = agent.invoke(state)

    #conversation_history = [{"role": "user", "content": msg} for msg in request.messages]
    #response = agent.invoke({"input": conversation_history})
    #return {"response": response["output"]}

    return result